# Feature Engineering

In this notebook, we will focus on creating new features from the raw dataset to improve the performance of our machine learning models.

In [39]:
# =============================================================================
# PHASE 2 : FEATURE ENGINEERING
# =============================================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Charger les données
train_data = pd.read_csv('../data/raw/train.csv')
test_data = pd.read_csv('../data/raw/test.csv')

# Vérifier les colonnes disponibles
print("📋 Colonnes disponibles:")
print(train_data.columns.tolist())
print(f"\nShape train: {train_data.shape}")
print(train_data.head())

📋 Colonnes disponibles:
['transaction_id', 'step', 'type', 'amount', 'customer_id', 'age', 'is_fraud']

Shape train: (30000, 7)
                         transaction_id  step      type  amount customer_id  \
0  38d8cd1a-ab64-48a8-982b-547db096f8a5   421   PAYMENT  311000   C13646530   
1  2440928e-8a4c-4e74-a9c5-6329668be13d   325   PAYMENT  399600    C9452763   
2  973c2a3c-5676-477c-ac30-81334b01e90d   565   PAYMENT    7500   C31260354   
3  5413d0f4-b4f0-406f-bd91-91de1dce573c   511  TRANSFER   94000    C8841991   
4  c9fa99e2-42c8-4dd8-94ba-aa3e44bfdd98    63  CASH_OUT   15000    C3176716   

   age  is_fraud  
0   24         0  
1   25         0  
2   38         0  
3   26         0  
4   26         0  


In [40]:
# =============================================================================
# 2.1 - VARIABLES TEMPORELLES
# =============================================================================

def create_temporal_features(df):
    """Créer les features temporelles basées sur 'step'"""
    
    # hour_of_day : heure dans la journée (0-23)
    df['hour_of_day'] = (df['step'] - 1) % 24
    
    # day_of_week : jour de la semaine (0=Lundi, 6=Dimanche)
    df['day_of_week'] = ((df['step'] - 1) // 24) % 7
    
    # week_number : numéro de la semaine
    df['week_number'] = (df['step'] - 1) // (24 * 7)
    
    # is_night : transaction de nuit (22h-6h)
    df['is_night'] = ((df['hour_of_day'] >= 22) | (df['hour_of_day'] <= 6)).astype(int)
    
    # is_weekend : samedi ou dimanche
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    # is_business_hours : heures de bureau (8h-17h)
    df['is_business_hours'] = df['hour_of_day'].between(8, 17).astype(int)
    
    return df

# Tester
train_data = create_temporal_features(train_data)
print("✅ 2.1 Variables temporelles créées")
print(train_data[['step', 'hour_of_day', 'day_of_week', 'is_night', 'is_weekend']].head(10))

✅ 2.1 Variables temporelles créées
   step  hour_of_day  day_of_week  is_night  is_weekend
0   421           12            3         0           0
1   325           12            6         0           1
2   565           12            2         0           0
3   511            6            0         1           0
4    63           14            2         0           0
5   468           11            5         0           1
6   706            9            1         0           0
7   602            1            4         1           0
8   183           14            0         0           0
9   373           12            1         0           0


In [41]:
# =============================================================================
# 2.2 - VARIABLES DE MONTANT
# =============================================================================

def create_amount_features(df):
    """Créer les features basées sur le montant"""
    
    # Transformations logarithmique et racine carrée
    df['amount_log'] = np.log1p(df['amount'])
    df['amount_sqrt'] = np.sqrt(df['amount'])
    
    # Calculer les quantiles sur le dataset
    Q95 = df['amount'].quantile(0.95)
    Q99 = df['amount'].quantile(0.99)
    
    print(f"  Q95 amount: {Q95:,.2f}")
    print(f"  Q99 amount: {Q99:,.2f}")
    
    # Indicateurs de montants élevés
    df['is_high_amount'] = (df['amount'] > Q95).astype(int)
    df['is_very_high_amount'] = (df['amount'] > Q99).astype(int)
    
    # Montant rond (multiple de 1000)
    df['is_round_amount'] = (df['amount'] % 1000 == 0).astype(int)
    
    return df

# Tester
train_data = create_amount_features(train_data)
print("\n✅ 2.2 Variables de montant créées")
print(train_data[['amount', 'amount_log', 'is_high_amount', 'is_round_amount']].head(10))

  Q95 amount: 550,015.00
  Q99 amount: 1,510,520.00

✅ 2.2 Variables de montant créées
   amount  amount_log  is_high_amount  is_round_amount
0  311000   12.647551               0                1
1  399600   12.898222               0                0
2    7500    8.922792               0                0
3   94000   11.451061               0                1
4   15000    9.615872               0                1
5  111800   11.624476               0                0
6   37000   10.518700               0                1
7   91500   11.424105               0                0
8  270000   12.506181               0                1
9   18500    9.825580               0                0


In [42]:
# =============================================================================
# 2.3 - VARIABLES DÉMOGRAPHIQUES (ÂGE)
# =============================================================================

def create_age_features(df):
    """Créer les features basées sur l'âge"""
    
    # Indicateurs d'âge
    df['is_senior'] = (df['age'] >= 60).astype(int)
    df['is_young'] = (df['age'] <= 25).astype(int)
    
    # Groupes d'âge
    df['age_group'] = pd.cut(
        df['age'], 
        bins=[0, 25, 40, 60, 100], 
        labels=['young', 'adult', 'middle', 'senior']
    )
    
    # Encoder age_group en numérique
    age_mapping = {'young': 0, 'adult': 1, 'middle': 2, 'senior': 3}
    df['age_group_encoded'] = df['age_group'].map(age_mapping)
    
    return df

# Tester
train_data = create_age_features(train_data)
print("✅ 2.3 Variables démographiques créées")
print(train_data[['age', 'is_senior', 'is_young', 'age_group']].head(10))

✅ 2.3 Variables démographiques créées
   age  is_senior  is_young age_group
0   24          0         1     young
1   25          0         1     young
2   38          0         0     adult
3   26          0         0     adult
4   26          0         0     adult
5   20          0         1     young
6   29          0         0     adult
7   22          0         1     young
8   47          0         0    middle
9   22          0         1     young


In [43]:
# =============================================================================
# 2.4 - ENCODAGE DES VARIABLES CATÉGORIELLES
# =============================================================================

# Créer et sauvegarder le LabelEncoder pour 'type'
type_encoder = LabelEncoder()
type_encoder.fit(train_data['type'])

# Afficher le mapping
print("📋 Mapping du type de transaction:")
for i, class_name in enumerate(type_encoder.classes_):
    print(f"  {class_name} → {i}")

def encode_type(df, encoder):
    """Encoder la variable 'type'"""
    df['type_encoded'] = encoder.transform(df['type'])
    return df

# Appliquer
train_data = encode_type(train_data, type_encoder)
print("\n✅ 2.4 Encodage catégoriel terminé")
print(train_data[['type', 'type_encoded']].drop_duplicates())

📋 Mapping du type de transaction:
  CASH_OUT → 0
  DEBIT → 1
  PAYMENT → 2
  TRANSFER → 3

✅ 2.4 Encodage catégoriel terminé
       type  type_encoded
0   PAYMENT             2
3  TRANSFER             3
4  CASH_OUT             0
7     DEBIT             1


In [44]:
# =============================================================================
# 2.5 - VARIABLES D'INTERACTION
# =============================================================================

def create_interaction_features(df):
    """Créer les features d'interaction"""
    
    # Montant par âge (éviter division par 0)
    df['amount_per_age'] = df['amount'] / (df['age'] + 1)
    
    # Interactions nuit x montant élevé
    df['night_high_amount'] = df['is_night'] * df['is_high_amount']
    
    # Interactions weekend x senior
    df['weekend_senior'] = df['is_weekend'] * df['is_senior']
    
    # Interactions nuit x senior
    df['night_senior'] = df['is_night'] * df['is_senior']
    
    # Bonus: autres interactions potentiellement utiles
    df['night_young'] = df['is_night'] * df['is_young']
    df['weekend_high_amount'] = df['is_weekend'] * df['is_high_amount']
    
    return df

# Tester
train_data = create_interaction_features(train_data)
print("✅ 2.5 Variables d'interaction créées")
print(train_data[['night_high_amount', 'weekend_senior', 'night_senior']].sum())

✅ 2.5 Variables d'interaction créées
night_high_amount    275
weekend_senior       903
night_senior         620
dtype: int64


In [45]:
# =============================================================================
# 2.6 - FONCTION GLOBALE
# =============================================================================

def create_all_features(df, encoder=None, fit_encoder=False):
    """
    Fonction globale qui applique tout le feature engineering
    
    Parameters:
    - df: DataFrame
    - encoder: LabelEncoder pour 'type' (pré-entraîné)
    - fit_encoder: Si True, crée un nouveau encoder
    
    Returns:
    - df: DataFrame avec features
    - encoder: LabelEncoder (si fit_encoder=True)
    """
    
    # 2.1 Variables temporelles
    df['hour_of_day'] = (df['step'] - 1) % 24
    df['day_of_week'] = ((df['step'] - 1) // 24) % 7
    df['week_number'] = (df['step'] - 1) // (24 * 7)
    df['is_night'] = ((df['hour_of_day'] >= 22) | (df['hour_of_day'] <= 6)).astype(int)
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['is_business_hours'] = df['hour_of_day'].between(8, 17).astype(int)
    
    # 2.2 Variables de montant
    df['amount_log'] = np.log1p(df['amount'])
    df['amount_sqrt'] = np.sqrt(df['amount'])
    Q95 = df['amount'].quantile(0.95)
    Q99 = df['amount'].quantile(0.99)
    df['is_high_amount'] = (df['amount'] > Q95).astype(int)
    df['is_very_high_amount'] = (df['amount'] > Q99).astype(int)
    df['is_round_amount'] = (df['amount'] % 1000 == 0).astype(int)
    
    # 2.3 Variables démographiques
    df['is_senior'] = (df['age'] >= 60).astype(int)
    df['is_young'] = (df['age'] <= 25).astype(int)
    df['age_group'] = pd.cut(df['age'], bins=[0, 25, 40, 60, 100], 
                             labels=['young', 'adult', 'middle', 'senior'])
    age_mapping = {'young': 0, 'adult': 1, 'middle': 2, 'senior': 3}
    df['age_group_encoded'] = df['age_group'].map(age_mapping)
    
    # 2.4 Encodage type
    if fit_encoder:
        encoder = LabelEncoder()
        df['type_encoded'] = encoder.fit_transform(df['type'])
    else:
        df['type_encoded'] = encoder.transform(df['type'])
    
    # 2.5 Variables d'interaction
    df['amount_per_age'] = df['amount'] / (df['age'] + 1)
    df['night_high_amount'] = df['is_night'] * df['is_high_amount']
    df['weekend_senior'] = df['is_weekend'] * df['is_senior']
    df['night_senior'] = df['is_night'] * df['is_senior']
    df['night_young'] = df['is_night'] * df['is_young']
    df['weekend_high_amount'] = df['is_weekend'] * df['is_high_amount']
    
    if fit_encoder:
        return df, encoder
    return df

print("✅ 2.6 Fonction globale create_all_features() créée")

✅ 2.6 Fonction globale create_all_features() créée


In [46]:
# =============================================================================
# APPLIQUER SUR TRAIN ET TEST
# =============================================================================

# Recharger les données propres
train_data = pd.read_csv('../data/raw/train.csv')
test_data = pd.read_csv('../data/raw/test.csv')

# Appliquer le feature engineering
train_data, type_encoder = create_all_features(train_data, fit_encoder=True)
test_data = create_all_features(test_data, encoder=type_encoder)

print("✅ Feature engineering appliqué sur train et test")
print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

✅ Feature engineering appliqué sur train et test
Train shape: (30000, 29)
Test shape: (10000, 28)


In [47]:
# =============================================================================
# 2.7 - NETTOYAGE FINAL
# =============================================================================

# Colonnes à supprimer
columns_to_drop = ['transaction_id', 'customer_id', 'type', 'age_group']

# Vérifier quelles colonnes existent avant de les supprimer
existing_cols_to_drop = [col for col in columns_to_drop if col in train_data.columns]
print(f"Colonnes à supprimer: {existing_cols_to_drop}")

# Supprimer
train_clean = train_data.drop(columns=existing_cols_to_drop, errors='ignore')
test_clean = test_data.drop(columns=existing_cols_to_drop, errors='ignore')

# Vérifier les NaN
print("\n🔍 Vérification des valeurs manquantes:")
print(train_clean.isnull().sum().sum(), "NaN au total dans train")
print(test_clean.isnull().sum().sum(), "NaN au total dans test")

# Remplacer les NaN si nécessaire
train_clean = train_clean.fillna(0)
test_clean = test_clean.fillna(0)

Colonnes à supprimer: ['transaction_id', 'customer_id', 'type', 'age_group']

🔍 Vérification des valeurs manquantes:
0 NaN au total dans train
0 NaN au total dans test


In [48]:
# Liste finale des features
features_columns = [col for col in train_clean.columns if col != 'is_fraud']

print("\n📋 LISTE FINALE DES FEATURES:")
print("=" * 50)
for i, col in enumerate(features_columns, 1):
    print(f"{i:2d}. {col}")
print("=" * 50)
print(f"Total: {len(features_columns)} features")


📋 LISTE FINALE DES FEATURES:
 1. step
 2. amount
 3. age
 4. hour_of_day
 5. day_of_week
 6. week_number
 7. is_night
 8. is_weekend
 9. is_business_hours
10. amount_log
11. amount_sqrt
12. is_high_amount
13. is_very_high_amount
14. is_round_amount
15. is_senior
16. is_young
17. age_group_encoded
18. type_encoded
19. amount_per_age
20. night_high_amount
21. weekend_senior
22. night_senior
23. night_young
24. weekend_high_amount
Total: 24 features


In [49]:
# =============================================================================
# SAUVEGARDER LES DONNÉES PROCESSED
# =============================================================================

import os
import pickle

# Créer le dossier si nécessaire
os.makedirs('../data/processed', exist_ok=True)

# Sauvegarder les datasets
train_clean.to_csv('../data/processed/train_engineered.csv', index=False)
test_clean.to_csv('../data/processed/test_engineered.csv', index=False)

# Sauvegarder l'encoder
with open('../data/processed/type_encoder.pkl', 'wb') as f:
    pickle.dump(type_encoder, f)

# Sauvegarder la liste des features
with open('../data/processed/features_columns.pkl', 'wb') as f:
    pickle.dump(features_columns, f)

print("✅ Fichiers sauvegardés:")
print("  - data/processed/train_engineered.csv")
print("  - data/processed/test_engineered.csv")
print("  - data/processed/type_encoder.pkl")
print("  - data/processed/features_columns.pkl")

✅ Fichiers sauvegardés:
  - data/processed/train_engineered.csv
  - data/processed/test_engineered.csv
  - data/processed/type_encoder.pkl
  - data/processed/features_columns.pkl


In [50]:
# =============================================================================
# RÉSUMÉ PHASE 2
# =============================================================================

print("\n" + "=" * 60)
print("📊 RÉSUMÉ PHASE 2 - FEATURE ENGINEERING")
print("=" * 60)
print(f"Features temporelles:    6 (hour_of_day, day_of_week, etc.)")
print(f"Features montant:        5 (amount_log, is_high_amount, etc.)")
print(f"Features démographiques: 4 (is_senior, is_young, age_group)")
print(f"Encodage catégoriel:     1 (type_encoded)")
print(f"Features interaction:    6 (night_high_amount, etc.)")
print(f"{'='*60}")
print(f"TOTAL FEATURES:          {len(features_columns)}")
print("=" * 60)
print("\n✅ PHASE 2 TERMINÉE!")


📊 RÉSUMÉ PHASE 2 - FEATURE ENGINEERING
Features temporelles:    6 (hour_of_day, day_of_week, etc.)
Features montant:        5 (amount_log, is_high_amount, etc.)
Features démographiques: 4 (is_senior, is_young, age_group)
Encodage catégoriel:     1 (type_encoded)
Features interaction:    6 (night_high_amount, etc.)
TOTAL FEATURES:          24

✅ PHASE 2 TERMINÉE!
